In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import json
import pickle
import argparse
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from process_data import save_pickle, load_pickle, load_task, load_glove_weights, to_var, make_word_vector, make_one_hot
from word_embedding import WordEmbedding
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import sys

In [3]:
sys.argv = ['a.py']

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=8, help='input batch size')
parser.add_argument('--embd_size', type=int, default=100, help='word embedding size')
args = parser.parse_args()

In [4]:
# train_data, train_ctx_maxlen = load_task('./dataset/train-v1.1.json')
# save_pickle(train_data, './pickles/train_data.pickle')
train_data = load_pickle('./pickles/train_data.pickle')
# dev_data, dev_ctx_maxlen = load_task('./dataset/dev-v1.1.json')
# save_pickle(dev_data, './pickles/dev_data.pickle')
dev_data = load_pickle('./pickles/dev_data.pickle')
data = train_data+dev_data
# ctx_maxlen = max(train_ctx_maxlen, dev_ctx_maxlen)
ctx_maxlen = 4063 #TODO
args.ctx_maxlen = ctx_maxlen
print('context char-level maxlen:', ctx_maxlen)

vocab, vocab_a = set(), set()
for ctx, _, query,answer, _, _ in data:
    vocab |= set(ctx + query)# + answer)
    vocab_a |= set(answer)
    
vocab = list(sorted(vocab))
vocab_a = list(sorted(vocab_a))
w2i = dict((w, i) for i, w in enumerate(vocab, 0))
i2w = dict((i, w) for i, w in enumerate(vocab, 0))
a2i = dict((w, i) for i, w in enumerate(vocab_a, 0))
i2a = dict((i, w) for i, w in enumerate(vocab_a, 0))
args.vocab_size = len(vocab)
args.ans_size = len(vocab_a)
print('vocab size', len(vocab))
print('ans size', len(vocab_a))

ctx_sent_maxlen = max([len(c) for c, _, _, _, _, _ in data])
query_sent_maxlen = max([len(q) for _, _, q, _, _, _ in data])
print('ctx_sent_maxlen:', ctx_sent_maxlen)
print('query_sent_maxlen:', query_sent_maxlen)
args.ctx_sent_maxlen = ctx_sent_maxlen
args.query_sent_maxlen = query_sent_maxlen

glove_embd = load_pickle('./pickles/glove_embd.pickle')
# glove_embd = torch.from_numpy(load_glove_weights('./dataset', args.embd_size, len(vocab), w2i)).type(torch.FloatTensor)
# save_pickle(glove_embd, './pickles/glove_embd.pickle')
args.pre_embd = glove_embd

load ./pickles/train_data.pickle
load ./pickles/dev_data.pickle
context char-level maxlen: 4063
vocab size 123295
ans size 78025
ctx_sent_maxlen: 766
query_sent_maxlen: 60
load ./pickles/glove_embd.pickle


In [ ]:
class JNet(nn.Module):
    def __init__(self, args):
        super(JNet, self).__init__()
        self.embd_size = args.embd_size
        h_emb = (int)(self.embd_size*0.5)
        
        self.embd = WordEmbedding(args)
        self.ctx_birnn = nn.ModuleList([nn.GRU(self.embd_size, h_emb, bidirectional=True, dropout=0.2) for _ in range(2)])
        self.query_birnn = nn.ModuleList([nn.GRU(self.embd_size, h_emb, bidirectional=True, dropout=0.2) for _ in range(2)])
        self.last_rnn = nn.GRU(self.embd_size, self.embd_size, bidirectional=True, dropout=0.2)
        self.last_layer = nn.Linear(args.ctx_sent_maxlen, args.ctx_maxlen)
        
    def forward(self, context, query):
        x = self.embd(context) # (N, JX, d)
        q_embd = self.embd(query) # (N, JQ, d)
        q, _h = self.ctx_birnn[0](q_embd) # (N, JQ, d)
        q = q.sum(1).unsqueeze(2) # (N, d, 1)

        for i in range(2):
            d, _h = self.ctx_birnn[i](x) # (N, JX, d)
        
            attn = F.softmax(torch.bmm(d, q).squeeze()) # (N, JX)
            attn_q = torch.bmm(attn.unsqueeze(2), q.transpose(2, 1)) # (N, JX, d)
            x = d * attn_q # (N, JX, d)
        x, _ = self.last_rnn(x) # (N, JX, 2d)
        x = x.sum(2) # (N, JX)
#         x = x.view(x.size(0), -1)
        x = F.log_softmax(self.last_layer(x))
        return x
        
def train(model, optimizer, loss_fn, n_epoch=5, batch_size=32):
    for epoch in range(n_epoch):
        print('---Epoch', epoch)
        for i in tqdm(range(0, len(data)-batch_size, batch_size)): # TODO shuffle, last elms
            batch_data = data[i:i+batch_size]
            c = [d[0] for d in batch_data]
            q = [d[2] for d in batch_data]
            context_var = make_word_vector(c, w2i, ctx_sent_maxlen)
            query_var = make_word_vector(q, w2i, query_sent_maxlen)
            labels = [d[4][0] for d in batch_data]
            labels = to_var(torch.LongTensor(labels))#.squeeze()) #
            outs = model(context_var, query_var)
            loss = loss_fn(outs, labels)
            print(loss.data[0])
            model.zero_grad()
            loss.backward()
            optimizer.step()
        
        _, preds = torch.max(outs, 1)
        ct = 0
        for pred, label in zip(preds, labels):
            if pred.data[0] == label.data[0]: 
                ct+=1
        print('Acc', ct, '/', len(preds))
        
    
model = JNet(args)
if torch.cuda.is_available():
    model.cuda()
# print(model)
# for p in model.parameters():
#     print(p)
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adadelta(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=0.5, weight_decay=0.999)
train(model, optimizer, loss_fn)
print('fin')


  0%|          | 3/3067 [00:00<02:11, 23.33it/s]

---Epoch 0
8.315591812133789
8.300823211669922
8.295954704284668
8.311386108398438



  0%|          | 9/3067 [00:00<02:08, 23.80it/s]

8.321048736572266
8.315695762634277
8.325821876525879
8.324726104736328
8.309561729431152



  0%|          | 15/3067 [00:00<02:03, 24.69it/s]

...

7.340137958526611
7.261232852935791
7.078273773193359
6.902990341186523
6.49388313293457
7.066873550415039


 41%|████      | 1245/3067 [00:45<01:02, 29.23it/s]

7.427045822143555
6.506670951843262
6.807179927825928
6.724869728088379
6.857159614562988
6.699617385864258
6.600242614746094


 41%|████      | 1251/3067 [00:45<01:03, 28.77it/s]

6.4417266845703125
6.777556419372559
6.859004020690918
6.375052452087402
7.2204670906066895
6.551926136016846
